In [1]:
!pip install pycaret

In [30]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from pycaret.classification import *
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("data.csv")
print(df.head())

    ad_id reporting_start reporting_end campaign_id fb_campaign_id    age  \
0  708746      17/08/2017    17/08/2017         916         103916  30-34   
1  708749      17/08/2017    17/08/2017         916         103917  30-34   
2  708771      17/08/2017    17/08/2017         916         103920  30-34   
3  708815      30/08/2017    30/08/2017         916         103928  30-34   
4  708818      17/08/2017    17/08/2017         916         103928  30-34   

  gender  interest1  interest2  interest3  impressions  clicks  spent  \
0      M         15         17         17       7350.0       1   1.43   
1      M         16         19         21      17861.0       2   1.82   
2      M         20         25         22        693.0       0   0.00   
3      M         28         32         32       4259.0       1   1.25   
4      M         28         33         32       4133.0       1   1.29   

   total_conversion  approved_conversion  
0               2.0                  1.0  
1           

In [31]:
print("\nИнформация по столбцам:")
print(data.info())


Информация по столбцам:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ad_id                1143 non-null   int64  
 1   reporting_start      1143 non-null   object 
 2   reporting_end        1143 non-null   object 
 3   campaign_id          1143 non-null   object 
 4   fb_campaign_id       1143 non-null   object 
 5   age                  1143 non-null   object 
 6   gender               1143 non-null   object 
 7   interest1            1143 non-null   int64  
 8   interest2            1143 non-null   int64  
 9   interest3            1143 non-null   int64  
 10  impressions          1143 non-null   float64
 11  clicks               1143 non-null   int64  
 12  spent                1143 non-null   float64
 13  total_conversion     761 non-null    float64
 14  approved_conversion  761 non-null    float64
dtypes: float64(4)

Подготовка данных

In [32]:
# Преобразуем даты в datetime
data['reporting_start'] = pd.to_datetime(data['reporting_start'], dayfirst=True)
data['reporting_end'] = pd.to_datetime(data['reporting_end'], dayfirst=True)

In [33]:
# Рассчитаем длительность кампании в днях
data['campaign_duration'] = (data['reporting_end'] - data['reporting_start']).dt.days + 1

In [34]:

# Рассчитаем CTR (click-through rate)
data['CTR'] = data['clicks'] / data['impressions']

In [35]:
# Рассчитаем CPC (cost per click)
data['CPC'] = data['spent'] / data['clicks']
data['CPC'] = data['CPC'].replace([np.inf, -np.inf], np.nan)  # Заменяем бесконечные значения на NaN

In [36]:
# Рассчитаем CPA (cost per approved conversion)
data['CPA'] = data['spent'] / data['approved_conversion']
data['CPA'] = data['CPA'].replace([np.inf, -np.inf], np.nan)  # Заменяем бесконечные значения на NaN

In [37]:
# Заполним пропущенные значения
data['CPC'] = data['CPC'].fillna(0)
data['CPA'] = data['CPA'].fillna(0)

In [38]:
# Удалим ненужные столбцы
data = data.drop(['ad_id', 'fb_campaign_id', 'reporting_start', 'reporting_end'], axis=1)

In [39]:
# Преобразуем категориальные признаки
data = pd.get_dummies(data, columns=['age', 'gender'], drop_first=True)

In [46]:
# Анализ целевой переменной для классификации
data['is_approved'] = (data['approved_conversion'] > 0).astype(int)
print("Распределение классов:")
print(data['is_approved'].value_counts())

Распределение классов:
is_approved
0    790
1    353
Name: count, dtype: int64


In [52]:
# Удалим строки, где approved_conversion встречается только один раз
value_counts = data['approved_conversion'].value_counts()
rare_values = value_counts[value_counts < 2].index
data = data[~data['approved_conversion'].isin(rare_values)]

In [58]:
data = data.dropna(subset=['approved_conversion'])

In [59]:
from pycaret.classification import *
s = setup(data, target = 'approved_conversion', session_id = 123)

,Description,Value
0,Session id,123
1,Target,approved_conversion
2,Target type,Multiclass
3,Target mapping,"0.0: 0, 1.0: 1, 2.0: 2, 3.0: 3, 4.0: 4, 5.0: 5, 6.0: 6, 7.0: 7, 8.0: 8, 14.0: 9"
4,Original data shape,"(756, 119)"
5,Transformed data shape,"(756, 121)"
6,Transformed train set shape,"(529, 121)"
7,Transformed test set shape,"(227, 121)"
8,Numeric features,12
9,Categorical features,1


In [60]:
# functional API
best = compare_models()

Processing:   0%|          | 0/65 [00:00<?, ?it/s]